# Module Type

##### 🟢 &nbsp; &nbsp; Basic

In [1]:
module type X = sig
    val x : int
end

module type Y = sig
    val y: int 
end

module M = struct
    let x = 10
    let y = 11
end

(* MX is of type X and can access x *)
module MX = (M : X) 
(* or *)
module MX : X = M
(* or *)

(* MY is of type Y and can access y *)
module MY = (M : Y) 
(* or *)
module MY : Y = M

(* Check if M is of type X or Y *)
module _ : X = M
module _ : Y = M

module type X = sig val x : int end


module type Y = sig val y : int end


module M : sig val x : int val y : int end


module MX : X


module MX : X


module MY : Y


module MY : Y


<br/>

##### 🟢 &nbsp; &nbsp; Sealing Module with Module Type

In [2]:
module type XY = sig
    val x : int
    val y : int
end

(* MXY is sealed at module type XY *)
module MXY : XY = struct
    let x = 10
    let y = 10
end

module type XY_TYPE = XY
module MXY : XY_TYPE = struct
    let x = 10
    let y = 10
end

module MXYZ : module type of MXY = struct
    let x = 10
    let y = 10
    let z = 10
end

module type XY = sig val x : int val y : int end


module MXY : XY


module type XY_TYPE = XY


module MXY : XY_TYPE


module MXYZ : XY_TYPE


<br/>

##### 🟢 &nbsp; &nbsp; Type constraint the `with` operator

In [3]:
module type T = sig
    type t
    val data : t
end

module M : T = struct
    type t = int
    let data = 10
end

let a : int = M.data (* error cause T.t is abstract type and not compitable with int *)

module type T = sig type t val data : t end


module M : T


error: compile_error

##### 🖋 &nbsp; Solutions to the above issue

1 - Don't seal the module M at type T.  We still can check if module M satisfies module type T.

In [4]:
module type T = sig
    type t
    val data : t
end

module M  = struct
    type t = int
    let data = 10
end ;;

module _ : T = M ;;

let a : int = M.data ;;

module type T = sig type t val data : t end


module M : sig type t = int val data : int end


val a : int = 10


2 - Using the `with` operator

In [5]:
module type T = sig
    type t
    val data : t
end

module M : T with type t = int = struct
    type t = int
    let data = 10
end ;;

let a : int = M.data ;;

(* or *)

module type R = T with type t = int

module M : R = struct
    type t = int
    let data = 10
end ;;

let a : int = M.data ;;

module type T = sig type t val data : t end


module M : sig type t = int val data : t end


val a : int = 10


module type R = sig type t = int val data : t end


module M : R


val a : int = 10


<br/>

##### 🟢 &nbsp; &nbsp; Private type abbreviation https://v2.ocaml.org/manual/privatetypes.html#start-section

In [6]:
module M : sig
    type t = private int
    val data : t
end = struct
    type t = int
    let data = 10
end ;;

let a : M.t :> int = M.data;;

module M : sig type t = private int val data : t end


val a : int = 10


<br/>

##### 🟢 &nbsp; &nbsp; Destructive Substitution ( := ) - replace abstract type with the concrete type provided in constraint

In [7]:
module type T = sig
    type item
    type t
    val data : t
end

module M : T with type item = int and type t =  int list = struct
    type item = int
    type t = item list
    let data = List.init 10 Fun.id
end

module MN : T with type item := int and type t := int list = struct
    type item = int
    type t = item list
    let data = List.init 10 Fun.id
end

module type T = sig type item type t val data : t end


module M : sig type item = int type t = int list val data : t end


module MN : sig val data : int list end


<br/>

##### 🟢 &nbsp; &nbsp; Module constraint with the `with` operator https://cs3110.github.io/textbook/chapters/modules/module_type_constraints.html

In [8]:
module type XY = sig
  type x
  type y
end

module type T = sig
  module A : XY
end

module B = struct
  type x = int
  type y = float
end

module C : T with module A = B = struct
  module A = struct
    type x = int
    type y = float
    let x = 42
  end  
end

module type XY = sig type x type y end


module type T = sig module A : XY end


module B : sig type x = int type y = float end


module C : sig module A : sig type x = int type y = float end end


<br/>

##### 🟢 &nbsp; &nbsp; Multiple constraints

In [9]:
module type T = sig
    type item
    type t
    val data : t
end

module M : T with type item = int and type t = int list = struct
    type item = int
    type t = item list
    let data = List.init 10 Fun.id
end

module type T = sig type item type t val data : t end


module M : sig type item = int type t = int list val data : t end


<br/>

##### 🟢 &nbsp; &nbsp; include

In [10]:
module type T = sig
    type item
    type t
    val data : t
end

module type COUNT_T = sig
    include T
    val count : int
end

module Vec : COUNT_T with type item = float and type t = float array = struct
    type item = float
    type t = item array
    let data = Array.init 10 float_of_int
    let count = Array.length data
end

module type T = sig type item type t val data : t end


module type COUNT_T = sig type item type t val data : t val count : int end


module Vec :
  sig type item = float type t = float array val data : t val count : int end


<br/>

##### 🟢 &nbsp; &nbsp; Recovering the type of a module https://v2.ocaml.org/manual/moduletypeof.html#start-section

In [11]:
module A = struct
    let pi = 3.14
end

module type Maths = sig
    include (module type of A)
end

module A : sig val pi : float end


module type Maths = sig val pi : float end


<br/>

##### 🟢 &nbsp; &nbsp; Sample Code

In [12]:
module type POINT = sig
    type item
    type t
    val make : item -> item -> t
    val getX : t -> item
    val getY : t -> item
    val add : t -> t -> t
    val substract : t -> t -> t
    val to_string : t -> string
end

module PointPairInt : POINT with type item := int and type t := int * int = struct
    type item = int
    type t = item * item
    let make x y = (x, y)
    let getX (x, _) = x
    let getY (_, y) = y
    let add (x1, y1) (x2, y2) = (x1 + x2, y1 + y2)
    let substract (x1, y1) (x2, y2) = (x1 - x2, y1 - y2)
    let to_string (x, y) = Printf.sprintf "(x = %d, y = %d)" x y
end;;

let open PointPairInt in
let p1 = make 1 1
and p2 = make 2 2 in
(add p1 p2) |> to_string;;

type point = { x : float; y : float };;
module PointRecordFloat : POINT with type item := float and type t := point = struct
    type item = float
    type t = point
    let make x y = { x; y }
    let getX p = p.x
    let getY p = p.y
    let add p1 p2 = { x = p1.x +. p2.x; y = p1.y +. p2.y } 
    let substract p1 p2 = { x = p1.x -. p2.x; y = p1.y -. p2.y } 
    let to_string p = Printf.sprintf "{ x = %F, y = %F }" p.x p.y
end;;

let open PointRecordFloat in
let p1 = make 1. 1.
and p2 = make 2. 2. in
(add p1 p2) |> to_string;;

module type POINT =
  sig
    type item
    type t
    val make : item -> item -> t
    val getX : t -> item
    val getY : t -> item
    val add : t -> t -> t
    val substract : t -> t -> t
    val to_string : t -> string
  end


module PointPairInt :
  sig
    val make : int -> int -> int * int
    val getX : int * int -> int
    val getY : int * int -> int
    val add : int * int -> int * int -> int * int
    val substract : int * int -> int * int -> int * int
    val to_string : int * int -> string
  end


- : string = "(x = 3, y = 3)"


type point = { x : float; y : float; }


module PointRecordFloat :
  sig
    val make : float -> float -> point
    val getX : point -> float
    val getY : point -> float
    val add : point -> point -> point
    val substract : point -> point -> point
    val to_string : point -> string
  end


- : string = "{ x = 3., y = 3. }"
